In [1]:
import os
os.chdir("/app")

In [46]:
import pandas as pd
from pathlib import Path
from nbgrader.api import Gradebook
from nbgrader.apps import NbGraderAPI
from traitlets.config import Config
from tqdm.auto import tqdm
import re
import os

In [62]:
COURSE_ID = "itmo_recsys_2025_spring"

In [63]:
config = Config()
config.CourseDirectory.course_id = COURSE_ID
config.ExecutePreprocessor.environment = {
    'DATA_PATH_2': '/path/to/data'
}

In [64]:
nbg = NbGraderAPI(config=config)
gradebook = nbg.gradebook

In [65]:
ASSIGNMENT = "hw_3"
NOTEBOOK = "knn"  # without .ipynb

In [66]:
os.environ["DATA_PATH_3"] = "aadfad"

# Collect

In [7]:
nbg.collect(ASSIGNMENT, update=False)

{'success': True,
 'log': "[INFO] Processing 2 submissions of 'hw_3' for course 'itmo_recsys_2025_spring'\n[WARNING] \n    /usr/local/share/nbgrader/exchange/itmo_recsys_2025_spring/inbound/LeftGoga+hw_3+2025-04-01T11:26:03 UTC+00 claims to be submitted by LeftGoga but is owned by dev; cheating attempt?\n    you may disable this warning by unsetting the option CollectApp.check_owner\n    \n[INFO] Collecting submission: LeftGoga hw_3\n[WARNING] \n    /usr/local/share/nbgrader/exchange/itmo_recsys_2025_spring/inbound/Mihail-Olegovich+hw_3+2025-04-01T11:26:01 UTC+00 claims to be submitted by Mihail-Olegovich but is owned by dev; cheating attempt?\n    you may disable this warning by unsetting the option CollectApp.check_owner\n    \n[INFO] Collecting submission: Mihail-Olegovich hw_3\n"}

# Autograde

In [53]:
student_ids_requested_to_grade = [
    # "Mihail-Olegovich",
    "LeftGoga",
]

In [54]:
students = nbg.get_students()
student_ids = [student["id"] for student in students]
len(student_ids)

47

In [55]:
autograded_student_ids = nbg.get_autograded_students(ASSIGNMENT)
type(autograded_student_ids), len(autograded_student_ids)

(set, 2)

In [56]:
not_autograded_student_ids = [s_id for s_id in student_ids if s_id not in autograded_student_ids]
len(not_autograded_student_ids)

45

In [57]:
set(student_ids_requested_to_grade) - set(not_autograded_student_ids)

{'LeftGoga'}

In [58]:
set(not_autograded_student_ids) - set(student_ids_requested_to_grade)

{'1MaxOn1',
 '7Askar7',
 'AlViKl',
 'ArtemIgnatenko',
 'BulatMaratovich',
 'Darinochka',
 'DmitryRedko',
 'ElenaSuhova',
 'EugenePWN',
 'Evgenii-Iurin',
 'FanisNgv',
 'GlebIsrailevich',
 'Kraagger-del',
 'Laitielly',
 'NuatStanskiy',
 'ONEPANTSU',
 'PaulRychkov',
 'Postironia1',
 'Qeshtir',
 'ShamilNur',
 'UsefulTornado',
 'V-slav-github',
 'alexbuyan',
 'anaaaiva',
 'dimajyg',
 'katimanova',
 'khrstln',
 'koshkidadanet',
 'limmark21',
 'martynov-dm',
 'n0tmyself',
 'nbelyaev1',
 'nbusko',
 'nikiduki',
 'nonodoubt',
 'nsgorbunov',
 'pavelkochkin1',
 'popov101',
 'qw1zzard',
 'savoskinsemyonq',
 'semyondipner',
 'smalda',
 'stasstaf',
 'timsmr',
 'wilfordaf'}

In [59]:
if student_ids_requested_to_grade is not None:
    student_ids_to_grade = student_ids_requested_to_grade
    force = True
else:
    student_ids_to_grade = not_autograded_student_ids
    force = False

In [60]:
f"{force=}", student_ids_to_grade

('force=True', ['LeftGoga'])

In [67]:
results = []
for student_id in tqdm(student_ids_to_grade):
    res = nbg.autograde(assignment_id=ASSIGNMENT, student_id=student_id, force=force, create=False)
    results.append(res)

  0%|          | 0/1 [00:00<?, ?it/s]

In [40]:
successes = [res for res in results if res["success"]]
failures = [res for res in results if not res["success"]]

graded = [res for res in successes if "[INFO] Autograding" in res["log"]]
skipped = [res for res in successes if res["log"].startswith("[INFO] Skipping")]


print("Total: ", len(results))
print("failures: ", len(failures))
print("successes: ", len(successes))
print("graded: ", len(graded))
print("skipped: ", len(skipped))

Total:  2
failures:  0
successes:  2
graded:  2
skipped:  0


# Get per-task scores

In [43]:
if student_ids_requested_to_grade:
    students_to_get_scores = student_ids_requested_to_grade
else:
    raise NotImplementedError()

In [44]:
scores = {}
for student_id in tqdm(students_to_get_scores):
    nb = gradebook.find_submission_notebook(NOTEBOOK, ASSIGNMENT, student_id)
    student_scores = {
        "total": {"actual": nb.score, "max": nb.max_score},
        "cells": [
            {"actual": g.score, "max": g.max_score}
            for g in nb.grades
        ]
    }
    scores[student_id] = student_scores

  0%|          | 0/2 [00:00<?, ?it/s]

In [45]:
scores

{'Mihail-Olegovich': {'total': {'actual': 0.0, 'max': 15.0},
  'cells': [{'actual': 0.0, 'max': 3.0},
   {'actual': 0.0, 'max': 2.0},
   {'actual': 0.0, 'max': 2.0},
   {'actual': 0.0, 'max': 3.0},
   {'actual': 0.0, 'max': 5.0}]},
 'LeftGoga': {'total': {'actual': 0.0, 'max': 15.0},
  'cells': [{'actual': 0.0, 'max': 3.0},
   {'actual': 0.0, 'max': 2.0},
   {'actual': 0.0, 'max': 2.0},
   {'actual': 0.0, 'max': 3.0},
   {'actual': 0.0, 'max': 5.0}]}}

# Make human-readable feedback

In [31]:
CELL_NAMES = [
    "precision_and_recall",  
    "ap_at_3_for_user_2", 
    "map_at_3", 
    "dcg_at_3_for_user_2", 
    "idcg_at_3_for_user_2", 
    "ndcg_at_3",
    "weighted_recall_correct",
    "weighted_recall_efficient",
    "model_A",
    "model_B"
]

**Скоры предполагаются целыми**

In [32]:
feedbacks = []
for student_id, student_scores in scores.items():
    total_score = student_scores["total"]
    feedback_scores = [f"total: {total_score['actual']:.0f} / {total_score['max']:.0f}"]
    for cell_name, cell_score in zip(CELL_NAMES, student_scores["cells"]):
        feedback_scores.append(f"{cell_name}: {cell_score['actual']:.0f} / {cell_score['max']:.0f}")
    feedback = {
        "student_id": student_id,
        "feedback": ", ".join(feedback_scores),
        "score": round(total_score["actual"])
    }
    feedbacks.append(feedback)

In [33]:
feedbacks

[{'student_id': 'savoskinsemyonq',
  'feedback': 'total: 9 / 15, precision_and_recall: 1 / 1, ap_at_3_for_user_2: 1 / 1, map_at_3: 1 / 1, dcg_at_3_for_user_2: 1 / 1, idcg_at_3_for_user_2: 1 / 1, ndcg_at_3: 0 / 1, weighted_recall_correct: 0 / 2, weighted_recall_efficient: 0 / 3, model_A: 2 / 2, model_B: 2 / 2',
  'score': 9},
 {'student_id': 'ONEPANTSU',
  'feedback': 'total: 0 / 15, precision_and_recall: 0 / 1, ap_at_3_for_user_2: 0 / 1, map_at_3: 0 / 1, dcg_at_3_for_user_2: 0 / 1, idcg_at_3_for_user_2: 0 / 1, ndcg_at_3: 0 / 1, weighted_recall_correct: 0 / 2, weighted_recall_efficient: 0 / 3, model_A: 0 / 2, model_B: 0 / 2',
  'score': 0}]

In [34]:
df = pd.DataFrame(feedbacks)
df

,student_id,feedback,score
0,savoskinsemyonq,"total: 9 / 15, precision_and_recall: 1 / 1, ap...",9
1,ONEPANTSU,"total: 0 / 15, precision_and_recall: 0 / 1, ap...",0


In [35]:
df["feedback"][0]

'total: 9 / 15, precision_and_recall: 1 / 1, ap_at_3_for_user_2: 1 / 1, map_at_3: 1 / 1, dcg_at_3_for_user_2: 1 / 1, idcg_at_3_for_user_2: 1 / 1, ndcg_at_3: 0 / 1, weighted_recall_correct: 0 / 2, weighted_recall_efficient: 0 / 3, model_A: 2 / 2, model_B: 2 / 2'

In [83]:
df.to_csv("hw_2_rescore_feedback.tsv", sep="\t", index=False)

# Generate feedback

In [18]:
if student_ids_requested_to_grade is not None:
    student_ids_to_gen_feedback = student_ids_requested_to_grade
    force = True
else:
    student_ids_to_gen_feedback = nbg.get_autograded_students(ASSIGNMENT)
    force = False

len(student_ids_to_gen_feedback)

10

In [19]:
for s_id in tqdm(student_ids_to_gen_feedback):
    nbg.generate_feedback(assignment_id=ASSIGNMENT, student_id=s_id, force=force)

  0%|          | 0/10 [00:00<?, ?it/s]

In [29]:
res = nbg.generate_feedback(assignment_id=ASSIGNMENT, student_id="AlViKl", force=True)

# Extract scores from feedback

In [20]:
def get_scores():
    scores = {}
    for student_id in os.listdir("/app/feedback"):
        # print(student_id)
        path = Path("/app/feedback") / student_id / ASSIGNMENT / "metrics.html"
        try:
            text = path.read_text()
        except FileNotFoundError as e:
            print(e)
            continue
        
        total_match = re.search(r"metrics \(Score: (\d+\.\d+ / \d+\.\d+)\)", text)
        total_score = total_match.groups()[0]
    
        cell_scores = re.findall(r'<li><a href="#cell-\w+?">Test cell</a> \(Score: (\d+.\d+ / \d+.\d+)\)</li>', text)    
    
        scores[student_id] = {"total": total_score, "cells": cell_scores}
    return scores

In [24]:
scores = get_scores()
len(scores)

42

In [25]:
if student_ids_requested_to_grade:
    scores = {s_id: s for s_id, s in scores.items() if s_id in student_ids_requested_to_grade}
len(scores)

6

In [26]:
scores.keys()

dict_keys(['FanisNgv', '1MaxOn1', 'dimajyg', 'nonodoubt', 'Kraagger-del', 'koshkidadanet'])

In [27]:
student_ids_requested_to_grade

['AlViKl',
 'savoskinsemyonq',
 'koshkidadanet',
 '1MaxOn1',
 'dimajyg',
 'ONEPANTSU',
 'nikiduki',
 'Kraagger-del',
 'FanisNgv',
 'nonodoubt']

In [18]:
def parse_scores(scores):
    total_scores = {}
    students_feedback = []
    for student_id, feedback in scores.items():
        student_score = int(float(feedback["total"].split(" /")[0]))
        total_scores[student_id] = student_score
        feedbacks = [f"total: {feedback['total']}"]
        for name, result_score in zip(names, feedback["cells"]):
            feedbacks.append(f"{name}: {result_score}")
        students_feedback.append({
            "student_id": student_id,
            "feedback": ", ".join(feedbacks),
            "score": student_score
        })
    return total_scores, students_feedback

In [19]:
total_scores, students_feedback = parse_scores(scores)

In [24]:
total_scores["stasstaf"], [sf for sf in students_feedback if sf["student_id"] == "stasstaf"]

(8,
 [{'student_id': 'stasstaf',
   'feedback': 'total: 8.0 / 15.0, precision_and_recall: 1.0 / 1.0, ap_at_3_for_user_2: 1.0 / 1.0, map_at_3: 1.0 / 1.0, dcg_at_3_for_user_2: 1.0 / 1.0, idcg_at_3_for_user_2: 0.0 / 1.0, ndcg_at_3: 0.0 / 1.0, weighted_recall_correct: 0.0 / 2.0, weighted_recall_efficient: 0.0 / 3.0, model_A: 2.0 / 2.0, model_B: 2.0 / 2.0',
   'score': 8}])

In [102]:
manual_feedback = [
    {"student_id": "ONEPANTSU",
     "feedback": "Invalid notebook error: https://github.com/ONEPANTSU/recommendation-service/blob/hw_2/notebooks/hw_2/metrics.ipynb",
     "score": 0,},
    {"student_id": "savoskinsemyonq",
     "feedback": "404 error: https://github.com/savoskinsemyonq/RecSys_ITMO/blob/hw_2/notebooks/hw_2/metrics.ipynb",
     "score": 0,
    },
]

In [103]:
import pandas as pd
df = pd.DataFrame(students_feedback+manual_feedback)
df.to_csv("hw2_feedback.csv", index=False)

In [104]:
df.head()

,student_id,feedback,score
0,ElenaSuhova,"total: 15.0 / 15.0, precision_and_recall: 1.0 ...",15
1,nbusko,"total: 8.0 / 15.0, precision_and_recall: 1.0 /...",8
2,Darinochka,"total: 15.0 / 15.0, precision_and_recall: 1.0 ...",15
3,7Askar7,"total: 15.0 / 15.0, precision_and_recall: 1.0 ...",15
4,NuatStanskiy,"total: 8.0 / 15.0, precision_and_recall: 1.0 /...",8


# Regrade (after tests update)

In [115]:
students_to_regrade = df[df["score"]<15]["student_id"].values
students_to_regrade = [x for x in students_to_regrade if x not in ("ONEPANTSU", "savoskinsemyonq")]

In [116]:
results = []
for student_id in tqdm(students_to_regrade):
    res = nbg.autograde(assignment_id=ASSIGNMENT, student_id=student_id, force=True, create=False)
    results.append(res)

  0%|          | 0/25 [00:00<?, ?it/s]

In [117]:
successes = [res for res in results if res["success"]]
failures = [res for res in results if not res["success"]]

graded = [res for res in successes if "[INFO] Autograding" in res["log"]]
skipped = [res for res in successes if res["log"].startswith("[INFO] Skipping")]


print("Total: ", len(results))
print("failures: ", len(failures))
print("successes: ", len(successes))
print("graded: ", len(graded))
print("skipped: ", len(skipped))

Total:  25
failures:  0
successes:  25
graded:  25
skipped:  0


In [118]:
autograded_student_ids = nbg.get_autograded_students(ASSIGNMENT)
type(autograded_student_ids), len(autograded_student_ids)

(set, 39)

In [119]:
for s_id in tqdm(autograded_student_ids):
    nbg.generate_feedback(assignment_id=ASSIGNMENT, student_id=s_id, force=True)

  0%|          | 0/39 [00:00<?, ?it/s]

In [128]:
scores = get_scores()

In [126]:
scores

In [129]:
total_scores, students_feedback = parse_scores(scores)

In [133]:
total_scores

{'ElenaSuhova': 15,
 'nbusko': 13,
 'Darinochka': 15,
 '7Askar7': 15,
 'NuatStanskiy': 8,
 'DmitryRedko': 15,
 'qw1zzard': 15,
 'EugenePWN': 13,
 'FanisNgv': 8,
 'n0tmyself': 10,
 'martynov-dm': 11,
 'popov101': 8,
 'smalda': 15,
 'katimanova': 13,
 'anaaaiva': 13,
 'khrstln': 6,
 'timsmr': 12,
 'wilfordaf': 11,
 'semyondipner': 15,
 'nsgorbunov': 15,
 'BulatMaratovich': 9,
 'Qeshtir': 10,
 'Postironia1': 8,
 'V-slav-github': 13,
 'alexbuyan': 13,
 'ArtemIgnatenko': 9,
 'nbelyaev1': 15,
 'Laitielly': 10,
 'LeftGoga': 15,
 'ShamilNur': 11,
 'Mihail-Olegovich': 13,
 'UsefulTornado': 8,
 'Evgenii-Iurin': 8,
 'PaulRychkov': 10,
 'GlebIsrailevich': 15,
 'pavelkochkin1': 11}

In [132]:
students_feedback

[{'student_id': 'ElenaSuhova',
  'feedback': 'total: 15.0 / 15.0, precision_and_recall: 1.0 / 1.0, ap_at_3_for_user_2: 1.0 / 1.0, map_at_3: 1.0 / 1.0, dcg_at_3_for_user_2: 1.0 / 1.0, idcg_at_3_for_user_2: 1.0 / 1.0, ndcg_at_3: 1.0 / 1.0, weighted_recall_correct: 2.0 / 2.0, weighted_recall_efficient: 3.0 / 3.0, model_A: 2.0 / 2.0, model_B: 2.0 / 2.0',
  'score': 15},
 {'student_id': 'nbusko',
  'feedback': 'total: 13.0 / 15.0, precision_and_recall: 1.0 / 1.0, ap_at_3_for_user_2: 0.0 / 1.0, map_at_3: 0.0 / 1.0, dcg_at_3_for_user_2: 1.0 / 1.0, idcg_at_3_for_user_2: 1.0 / 1.0, ndcg_at_3: 1.0 / 1.0, weighted_recall_correct: 2.0 / 2.0, weighted_recall_efficient: 3.0 / 3.0, model_A: 2.0 / 2.0, model_B: 2.0 / 2.0',
  'score': 13},
 {'student_id': 'Darinochka',
  'feedback': 'total: 15.0 / 15.0, precision_and_recall: 1.0 / 1.0, ap_at_3_for_user_2: 1.0 / 1.0, map_at_3: 1.0 / 1.0, dcg_at_3_for_user_2: 1.0 / 1.0, idcg_at_3_for_user_2: 1.0 / 1.0, ndcg_at_3: 1.0 / 1.0, weighted_recall_correct: 2.0 /

In [130]:
df = pd.DataFrame(students_feedback+manual_feedback)
df.to_csv("hw2_feedback.csv", index=False)

In [131]:
df

,student_id,feedback,score
0,ElenaSuhova,"total: 15.0 / 15.0, precision_and_recall: 1.0 ...",15
1,nbusko,"total: 13.0 / 15.0, precision_and_recall: 1.0 ...",13
2,Darinochka,"total: 15.0 / 15.0, precision_and_recall: 1.0 ...",15
3,7Askar7,"total: 15.0 / 15.0, precision_and_recall: 1.0 ...",15
4,NuatStanskiy,"total: 8.0 / 15.0, precision_and_recall: 1.0 /...",8
5,DmitryRedko,"total: 15.0 / 15.0, precision_and_recall: 1.0 ...",15
6,qw1zzard,"total: 15.0 / 15.0, precision_and_recall: 1.0 ...",15
7,EugenePWN,"total: 13.0 / 15.0, precision_and_recall: 1.0 ...",13
8,FanisNgv,"total: 8.0 / 15.0, precision_and_recall: 1.0 /...",8
9,n0tmyself,"total: 10.0 / 15.0, precision_and_recall: 1.0 ...",10
